# Detección de Fuentes Discretas SUPPoSE
Este notebook permite aplicar un umbral dinámico sobre un campo continuo ρ(x, y) y visualizar las fuentes discretas resultantes.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from skimage.feature import peak_local_max
from skimage.draw import disk
from ipywidgets import interact, FloatSlider

In [ ]:

def generar_anillo(shape=(128, 128), radio_ext=40, grosor=10):
    imagen = np.zeros(shape, dtype=np.float32)
    centro = (shape[0] // 2, shape[1] // 2)
    rr_out, cc_out = disk(centro, radio_ext)
    rr_in, cc_in = disk(centro, radio_ext - grosor)
    imagen[rr_out, cc_out] = 1.0
    imagen[rr_in, cc_in] = 0.0
    return imagen

np.random.seed(42)
image_size = 128
rho = np.random.rand(image_size, image_size)
rho *= generar_anillo((image_size, image_size))


In [ ]:

def visualizar_fuentes_discretas(umbral_relativo=0.75):
    umbral_valor = umbral_relativo * np.max(rho)
    mascara = rho > umbral_valor
    maximos = peak_local_max(rho, min_distance=3, threshold_abs=umbral_valor, labels=mascara)
    rho_discreta = np.zeros_like(rho)
    for y, x in maximos:
        rho_discreta[y, x] = rho[y, x]

    fig, ax = plt.subplots(figsize=(6, 6))
    ax.imshow(rho, cmap='gray')
    ax.scatter(maximos[:, 1], maximos[:, 0], color='red', s=20, label='Fuentes discretas')
    ax.set_title(f"Fuentes SUPPoSE (umbral {int(umbral_relativo*100)}%)")
    ax.axis('off')
    ax.legend()
    plt.show()


In [ ]:
interact(visualizar_fuentes_discretas, umbral_relativo=FloatSlider(value=0.75, min=0.1, max=0.99, step=0.01))

## Exportar coordenadas de fuentes discretas a CSV

In [ ]:

import pandas as pd

def exportar_fuentes_csv(rho, umbral_relativo=0.75, nombre_archivo="fuentes_discretas.csv"):
    umbral_valor = umbral_relativo * np.max(rho)
    mascara = rho > umbral_valor
    maximos = peak_local_max(rho, min_distance=3, threshold_abs=umbral_valor, labels=mascara)
    intensidades = [rho[y, x] for y, x in maximos]
    df = pd.DataFrame(maximos, columns=["y", "x"])
    df["intensidad"] = intensidades
    df.to_csv(nombre_archivo, index=False)
    print(f"Exportado a {nombre_archivo}")


In [ ]:
exportar_fuentes_csv(rho, umbral_relativo=0.75)

## Reconstrucción desde archivo CSV

In [ ]:

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy.ndimage import gaussian_filter

def reconstruir_desde_csv(nombre_archivo, shape=(128, 128), sigma=2.0, plot=True):
    df = pd.read_csv(nombre_archivo)
    reconstruida = np.zeros(shape, dtype=np.float32)
    for _, fila in df.iterrows():
        y, x = int(fila['y']), int(fila['x'])
        if 0 <= y < shape[0] and 0 <= x < shape[1]:
            reconstruida[y, x] = fila['intensidad']
    if sigma > 0:
        reconstruida = gaussian_filter(reconstruida, sigma=sigma)
    if plot:
        plt.figure(figsize=(5, 5))
        plt.imshow(reconstruida, cmap='gray')
        plt.title("Reconstrucción desde fuentes CSV")
        plt.axis('off')
        plt.show()
    return reconstruida


In [ ]:
reconstruir_desde_csv("fuentes_discretas.csv", shape=(128,128), sigma=2.0)